In [1]:
# Sin: I used a newer version of pgmpy than reflected in the book

import pgmpy
print(pgmpy.__version__)

1.0.0


In [2]:
from pgmpy.factors.discrete import DiscreteFactor
dist = DiscreteFactor(
    variables=["X"],
    cardinality=[3],
    values=[.45, .30, .25],
    state_names={"X": ["1", "2", "3"]}
)
print(dist)

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


In [3]:
joint = DiscreteFactor(
    variables=["X", "Y"],
    cardinality=[3, 2],
    values=[.25, .2, .2, .1, .15, .1],
    state_names={
        "X": ["1", "2", "3"],
        "Y": ["0", "1"]
    }
)
print(joint)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(0) |     0.2500 |
+------+------+------------+
| X(1) | Y(1) |     0.2000 |
+------+------+------------+
| X(2) | Y(0) |     0.2000 |
+------+------+------------+
| X(2) | Y(1) |     0.1000 |
+------+------+------------+
| X(3) | Y(0) |     0.1500 |
+------+------+------------+
| X(3) | Y(1) |     0.1000 |
+------+------+------------+


In [4]:
# Summing over or "marginalizing" returns the original distribution from Cell 1
print(joint.marginalize(variables=["Y"], inplace=False))

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


In [5]:
# Marginalizing over X gives us the marginal distribution of Y
print(joint.marginalize(variables=["X"], inplace=False))

+------+----------+
| Y    |   phi(Y) |
+======+==========+
| Y(0) |   0.6000 |
+------+----------+
| Y(1) |   0.4000 |
+------+----------+


In [6]:
# Using the joint distribution and the distribution of X, we can derive the conditional probability of Y given X
print(joint/dist)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(0) |     0.5556 |
+------+------+------------+
| X(1) | Y(1) |     0.4444 |
+------+------+------------+
| X(2) | Y(0) |     0.6667 |
+------+------+------------+
| X(2) | Y(1) |     0.3333 |
+------+------+------------+
| X(3) | Y(0) |     0.6000 |
+------+------+------------+
| X(3) | Y(1) |     0.4000 |
+------+------+------------+


In [7]:
from pgmpy.factors.discrete.CPD import TabularCPD

# Single variable conditioinal distribution table
PYgivenX = TabularCPD(
    variable="Y",
    variable_card=2,
    values=[
        [.25/.45, .20/.30, .15/.25],
        [.20/.45, .10/.30, .10/.25]
    ],
    evidence=["X"],
    evidence_card=[3],
    state_names={
        "X": ["1", "2", "3"],
        "Y": ["0", "1"]
    }
)
print(PYgivenX)

+------+--------------------+---------------------+------+
| X    | X(1)               | X(2)                | X(3) |
+------+--------------------+---------------------+------+
| Y(0) | 0.5555555555555556 | 0.6666666666666667  | 0.6  |
+------+--------------------+---------------------+------+
| Y(1) | 0.4444444444444445 | 0.33333333333333337 | 0.4  |
+------+--------------------+---------------------+------+


Markovian assumption, that the value in one node is determined only by the value of the node directly before it, will play heavily in the models we study.

## Canonical Parameters in Pyro Example

In [8]:
import torch
from pyro.distributions import Bernoulli, Categorical, Gamma, Normal

print(Categorical(probs=torch.tensor([.45, .30, .25])))
print(Normal(loc=0.0, scale=1.0))
print(Bernoulli(probs=0.4))
print(Gamma(concentration=1.0, rate=2.0))

Categorical(probs: torch.Size([3]))
Normal(loc: 0.0, scale: 1.0)
Bernoulli(probs: 0.4000000059604645)
Gamma(concentration: 1.0, rate: 2.0)


/Users/hemphillmc/ds_playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
bern = Bernoulli(probs=0.4)
bern

Bernoulli(probs: 0.4000000059604645)

In [10]:
# Note that we typically work with log probabilities for computational advantages, but can switch back
# to normal probabilities with the .exp() method

lprob = bern.log_prob(torch.tensor(1.0))
lprob

tensor(-0.9163)

In [11]:
import math
print(math.exp(lprob))

0.40000001257540685


There is a massive, complexity reducing power to finding which variables in a distribution are independent of each other

The book outlines independence and conditional indepence

For building classifiers and other ML models, it would seem that understanding conditional depencence between groupings of variables and outcomes could help decide which variables to drop out to drive towards more efficient performance

Blood type example on page 38 gives a concrete visual

Chain Rule:

P(A, B) = P(A) * P(B|A)

P(A, B, C) = P(A) * P(B|A) * P(C|A,B)

## 2.2 Computational Probability

 How do we get deterministic machines to represent random processes?

In [12]:
# Simulating random variates in pgmpy

from pgmpy.factors.discrete import DiscreteFactor
dist = DiscreteFactor(
    variables=["Accepting Patients"],
    cardinality=[3],
    values=[.9, .05, .05],
    state_names={"Accepting Patients": ["Yes", "No", "Existing Roster Only"]}
)

dist.sample(n=10)

,Accepting Patients
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes
5,Yes
6,Yes
7,Yes
8,Yes
9,Yes


In [ ]:
joint = DiscreteFactor(
    variables=["Accepting Patients", "Is PCP"],
    cardinality=[3, 2],
    values=[.25, .2, .2, .1, .15, .1],
    state_names={
        "Accepting Patients": ["Yes", "No", "Existing Roster Only"],
        "Is PCP": ["Yes", "No"]e
    }
)
print(joint) 
print(joint.sample(n=10))

+------------------------------------------+-------------+----------------------------------+
| Accepting Patients                       | Is PCP      |   phi(Accepting Patients,Is PCP) |
+==========================================+=============+==================================+
| Accepting Patients(Yes)                  | Is PCP(Yes) |                           0.2500 |
+------------------------------------------+-------------+----------------------------------+
| Accepting Patients(Yes)                  | Is PCP(No)  |                           0.2000 |
+------------------------------------------+-------------+----------------------------------+
| Accepting Patients(No)                   | Is PCP(Yes) |                           0.2000 |
+------------------------------------------+-------------+----------------------------------+
| Accepting Patients(No)                   | Is PCP(No)  |                           0.1000 |
+------------------------------------------+-------------+--

In [14]:
# Pyro has sampling method as well
from pyro.distributions import Categorical
Categorical(probs=torch.tensor([.45, .3, .25])).sample()

tensor(1)

In [15]:
# Creating random processes in pgmpy and pyro

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.sampling import BayesianModelSampling

PZ = TabularCPD(
    variable='Z',
    variable_card=2,
    values=[[.65], [.35]],
    state_names = {
        'Z': ['0', '1']
    }
)

PXgivenZ = TabularCPD(
    variable='X',
    variable_card=2,
    values=[
        [.8, .6],
        [.2, .4]
    ],
    evidence=['Z'],
    evidence_card=[2],
    state_names = {
        'X': ['0', '1'],
        'Z': ['0', '1']
    }
)

PYgivenX = TabularCPD(
    variable='Y',
    variable_card=3,
    values=[
        [.1, .8],
        [.2, .1],
        [.7, .1]
    ],
    evidence=['X'],
    evidence_card=[2],
    state_names = {
        'Y': ['0', '1', '2'],
        'X': ['0', '1']
    }
)

In [16]:
model = DiscreteBayesianNetwork([('Z', 'X'), ('X', 'Y')])
model.add_cpds(PZ, PXgivenZ, PYgivenX)

generator = BayesianModelSampling(model)
generator.forward_sample(size=10)

Generating for node: Y: 100%|██████████| 3/3 [00:00<00:00, 142.95it/s]


,Z,X,Y
0,1,0,2
1,0,0,2
2,1,1,0
3,0,0,1
4,1,1,2
5,0,0,0
6,0,0,2
7,1,1,0
8,0,1,0
9,0,0,2


### 2.2.6 - Working with combinations of canonical distributions in Pyro

In [17]:
# Same probability relationship as above p(x, y, z), but different distributions underlying

import torch
from pyro.distributions import Bernoulli, Poisson, Gamma

In [18]:
z = Gamma(7.5, 1.0).sample()
x = Poisson(z).sample()
y = Bernoulli(x / (5 + x)).sample()

print(z, x, y)

tensor(3.5507) tensor(4.) tensor(1.)


In [19]:
# Coffee Shop Simulation
bean_quality = Gamma(10.5, 1.0).sample()
customer_count = Poisson(bean_quality).sample()
croissants_sold = Bernoulli(customer_count / (5 + customer_count)).sample()
print(f"Bean Score: {bean_quality}, Customer Count: {customer_count}, Croissants Sold: {croissants_sold}")

Bean Score: 16.011577606201172, Customer Count: 17.0, Croissants Sold: 1.0


### 2.2.7 - Random Processes with nuanced control flow in Pyro

In [20]:
import torch
from pyro.distributions import Bernoulli, Poisson, Gamma

bean_quality = Gamma(7.5, 1.0).sample()
customers = Poisson(bean_quality).sample()
scones_sold = torch.tensor(0.0)

for i in range(int(customers)):
    scones_sold += Bernoulli(.5).sample()  # Control flow implementation

print(f"Bean Score: {bean_quality}, Customers: {customers}, Scones Sold: {scones_sold}")

Bean Score: 7.9351420402526855, Customers: 6.0, Scones Sold: 3.0


### Functions for Random Processes

In [21]:
import torch
import pyro

def random_process():
    z = pyro.sample("z", Gamma(7.5, 1.0))
    x = pyro.sample("x", Poisson(z))
    y = torch.tensor(0.0)

    for i in range(int(x)):
        y += pyro.sample(f"y{i}", Bernoulli(.5))
    
    return y

In [22]:
random_process()

tensor(5.)

In [23]:
generated_samples = torch.stack([random_process() for _ in range(100)])
generated_samples.mean()

tensor(3.7000)

In [24]:
torch.square(generated_samples).mean()

tensor(18.4600)

In [25]:
generated_samples

tensor([ 1.,  6.,  6.,  3.,  3.,  4.,  1.,  6.,  4.,  8.,  3.,  3.,  4.,  3.,
         3.,  2.,  2.,  1.,  1.,  5.,  2.,  4.,  4.,  1.,  9.,  5.,  1.,  0.,
         5.,  5.,  5.,  3.,  5.,  2.,  4.,  5.,  1.,  3.,  1.,  3.,  6., 12.,
         4.,  3.,  5.,  8.,  6.,  7.,  1.,  4.,  5.,  0.,  6.,  4.,  2.,  4.,
         6.,  5.,  0.,  4.,  2.,  4.,  3.,  4.,  3.,  4.,  2.,  7.,  4.,  5.,
         3.,  7.,  3.,  3.,  2.,  0.,  0.,  3.,  4.,  2.,  6.,  2.,  4.,  3.,
         6.,  3.,  1.,  4.,  3.,  0.,  5.,  4.,  3.,  3.,  3.,  5.,  4.,  2.,
         5., 10.])

## 2.2.9 Monte Carlo Estimation in Python
Code example here explores estimating "Bean Quality" given a certain number of scone sales.  By filtering for a certain number of scone sales, we're able to estimate the bean quality given that amount

In [27]:
import torch
import pyro
from pyro.distributions import Bernoulli, Gamma, Poisson

In [ ]:
def random_process():
    z = pyro.sample("z", Gamma(7.5, 1.0))  # webinar quality
    x = pyro.sample("x", Poisson(z))  # potential customers
    y = torch.tensor(0.0)

    for i in range(int(x)):
        y += pyro.sample(f"{i}", Bernoulli(.04))  # scones sold
    
    return z, y  # now returning both z (bean quality) and y (scones sold)

In [31]:
generated_samples = [random_process() for _ in range(1000)]
z_frame = torch.stack([z for z, _ in generated_samples])
z_frame.mean()

tensor(7.4298)

In [33]:
z_given_y = torch.stack([z for z, y in generated_samples if y == 3])
print(z_given_y.mean())

tensor(7.2177)


### "Formal" method definitions - 

 - <u>variable elimination</u>: summing over columns or rows of a probability table such that the resulting table represents a target distribution
 - <u>belief propogation</u>: passing a value back and forth between "neighborhoods" of values, sampling the results, and observing for convergence
 - <u>Variational Inference</u>: Approximating a distribution to represent a target distribution, but using gradient descent methods common deep learning. This is a core feature of Pyro
 - <u>mcmc</u> approaches produce chains of outcomes defined by parameters and eventually converge on declared distributions

## Data, Populations, Statistics, & Models

 - Opens the section by talking about interventions and how their impact on a given population may vary

Generating IID Samples

In [ ]:
# pgmpy
generator.forward_sample(size=10)

Generating for node: Y: 100%|██████████| 3/3 [00:00<00:00, 1334.49it/s]


,Z,X,Y
0,0,1,2
1,0,0,2
2,0,0,0
3,1,0,0
4,0,0,2
5,0,1,0
6,1,1,0
7,0,0,2
8,1,1,1
9,0,0,2


In [35]:
# pyro
import pyro
from pyro.distributions import Bernoulli, Poisson, Gamma

def model():
    z = pyro.sample("z", Gamma(7.5, 1.0))
    x = pyro.sample("x", Poisson(z))
    with pyro.plate("IID", 10):  # plate is a context manager for generating conditionally independent samples
        y = pyro.sample("y", Bernoulli(x / (5 + x)))
    return y

model()

tensor([1., 1., 0., 0., 1., 0., 1., 0., 1., 1.])

### "This is not a cigar" 
...but in relation to how distributions model populations. Robert was emphatic not to mistake an observed distribution for a definition of the population itself

Important distinctino becasue we want to focus on real world implications, not just observed statistical trends

To really highlight this, in the rock throwing example, he asked us to speculate on the null cases on combo of outcomes not observed

Super important distinction, as the null cases may not be represented in observed data but may impact distributions when observed.

We wouldn't want to treat unobserved variables as probability 0

#### Latent Variables

Latent variables are the outcomes that are not directly observed. Think how a corpus of documents may not have topics explicitly defined, but charactaristics in co-occurence of words implies topics

In [36]:
# Example data generating process

def true_dgp(jenny_inclination, brian_inclination, window_strength):  # cool, we are naming phenomena
    jenny_throws_rock = jenny_inclination > 0.5
    brian_throws_rock = brian_inclination > 0.5

    strength_of_impact = 0.0

    if jenny_throws_rock and brian_throws_rock:
        strength_of_impact = 0.8
    elif jenny_throws_rock or brian_throws_rock:
        strength_of_impact = 0.6
    window_breaks = window_strength < strength_of_impact

    return jenny_throws_rock, brian_throws_rock, window_breaks

# This is simply an illustration of a causal process that generates data


#### Why causal inference is hard

Multiple DGPs could result in the same observed distribution